In [1]:
import gym, gym.spaces
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
#other parameters are randomize and won't do customization
#hidden layers neurons number, number of episodes for every loop
#filter percentage for the best episodes(we will take the best 30%)
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

In [4]:
#We define 2 helper class to create 2 namedtuple.
#EpisodeStep store 1 step in a episode, it also stored observation from environment and the action performed
#Episode is the set of EpisodeStep, it store the non-discounted reward in one whole episode
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    #batch to save result of process, set a reward counter for the current episode
    #reset environment and construct softmax layer, it transfer the output to action probabilites
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    
    #every loop will convert observation to pytorch tensor and send to network to get the action probability
    #nn.Module will get observation value from the CartPole 1 * 4 Tensor
    #Because we didn't use softmax function at final layer, it will output original value
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        #we use tensor.data to uncompress tensor and convert to Numpy array, this has same 2 dimension as input data
        #we want to get the first batch from the batch array, therefore we use [0], which is action probabilities
        act_probs = act_probs_v.data.numpy()[0]
        
        #we use random.choice() to sample from action probabilities, and put the action to environment to get next 
        #observation, reward and episode is done or not
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        
        #we add the reward to total reward, we store the observation and action pair into the episode_steps, the 
        #observation is before action, not after
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        
        #when gameover, episode is done, we will append the reward to total reward and reset environment and episode rewards
        #if batch accumulates enough episodes,we yield the result to caller for further process
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        
        #get observation value from environment to current observation variable
        #repeat everything after: pass observation result to network, sample action and take action, let environment
        #handle action, save result
        obs = next_obs

In [6]:
#training loop
def filter_batch(batch, percentile):
    #we use numpy percentile to caluclate the reward_bound with the batch reward we got
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    #reward_mean for monitoring
    reward_mean = float(np.mean(rewards))
    
    #if reward is larger than reward boundary, we keep it and store the observation and action
    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))
        
    #transform observation and action to vector and put in array, last 2 will just put in TensorBoard for monitoring but 
    # no real use
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)
        
    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

# The reward cannot converge because the environment and reward mechanism is very different, the reward will mark as success(1) or fail(0), and we only have very little chance by random walking will go to the goal. So if we use >50-70% as elite episode for training, probably there are many failed episodes


# To conclude, if using cross entropy, the episode is shorter, the result is better. The total reward should be able to divide the good and bad episode. Also before success, there won't be any indication of whether the target is going to achieve as good episode or bad episode.

In [8]:
if __name__ == "__main__":
    #we create all necessary objects: environment, network, target function, optimizer and TensorBoard writer
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))
    #The line below will create a monitor to save the agent action as video
    env = gym.wrappers.Monitor(env, directory="mon2", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n
    
    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-frozenlake-naive")
    
    #we get the batch in loop, filter the batch, get observation and action vector, reward boundary and mean
    #we make the gradient zero and give the observation to network and get the action score
    #action score will put to target function to calculate the difference between network output and agent chosen action
    #so the agent will choose the action will higher network output value
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        
        #For monitoring, show iterate number, loss, batch reward mean, reward boundary, all these values will be 
        #written in TensorBoard
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" %(iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        
        #if mean reward > 199, we stop training, it is because in Gym, when 100 episodes > 195,
        #the cartpole problem is said to be successfully solved, it can balance infinitely long,
        #but in CartPole environment, it used TimeLimit to limit the episodes within 200, so it is forced to stop after
        #200 steps. Therefore we use > 199 steps as to indicate the problem solved
        if reward_m > 199:
            print("Solved!")
            break
    writer.close()

0: loss=1.383, reward_mean=0.1, reward_bound=0.0
1: loss=1.413, reward_mean=0.0, reward_bound=0.0
2: loss=1.401, reward_mean=0.0, reward_bound=0.0
3: loss=1.383, reward_mean=0.0, reward_bound=0.0
4: loss=1.357, reward_mean=0.0, reward_bound=0.0
5: loss=1.355, reward_mean=0.0, reward_bound=0.0
6: loss=1.406, reward_mean=0.0, reward_bound=0.0
7: loss=1.382, reward_mean=0.0, reward_bound=0.0
8: loss=1.376, reward_mean=0.1, reward_bound=0.0
9: loss=1.355, reward_mean=0.0, reward_bound=0.0
10: loss=1.377, reward_mean=0.1, reward_bound=0.0
11: loss=1.376, reward_mean=0.1, reward_bound=0.0
12: loss=1.382, reward_mean=0.0, reward_bound=0.0
13: loss=1.310, reward_mean=0.1, reward_bound=0.0
14: loss=1.346, reward_mean=0.0, reward_bound=0.0
15: loss=1.351, reward_mean=0.0, reward_bound=0.0
16: loss=1.337, reward_mean=0.0, reward_bound=0.0
17: loss=1.390, reward_mean=0.1, reward_bound=0.0
18: loss=1.371, reward_mean=0.0, reward_bound=0.0
19: loss=1.317, reward_mean=0.0, reward_bound=0.0
20: loss=1

169: loss=0.001, reward_mean=0.1, reward_bound=0.0
170: loss=0.001, reward_mean=0.0, reward_bound=0.0
171: loss=0.001, reward_mean=0.1, reward_bound=0.0
172: loss=0.001, reward_mean=0.1, reward_bound=0.0
173: loss=0.078, reward_mean=0.1, reward_bound=0.0
174: loss=0.001, reward_mean=0.1, reward_bound=0.0
175: loss=0.001, reward_mean=0.0, reward_bound=0.0
176: loss=0.001, reward_mean=0.1, reward_bound=0.0
177: loss=0.001, reward_mean=0.1, reward_bound=0.0
178: loss=0.001, reward_mean=0.1, reward_bound=0.0
179: loss=0.001, reward_mean=0.1, reward_bound=0.0
180: loss=0.001, reward_mean=0.1, reward_bound=0.0
181: loss=0.001, reward_mean=0.0, reward_bound=0.0
182: loss=0.000, reward_mean=0.1, reward_bound=0.0
183: loss=0.001, reward_mean=0.0, reward_bound=0.0
184: loss=0.001, reward_mean=0.0, reward_bound=0.0
185: loss=0.001, reward_mean=0.1, reward_bound=0.0
186: loss=0.001, reward_mean=0.0, reward_bound=0.0
187: loss=0.001, reward_mean=0.1, reward_bound=0.0
188: loss=0.001, reward_mean=0.

331: loss=0.000, reward_mean=0.0, reward_bound=0.0
332: loss=0.000, reward_mean=0.1, reward_bound=0.0
333: loss=0.000, reward_mean=0.0, reward_bound=0.0
334: loss=0.000, reward_mean=0.1, reward_bound=0.0
335: loss=0.000, reward_mean=0.1, reward_bound=0.0
336: loss=0.000, reward_mean=0.1, reward_bound=0.0
337: loss=0.000, reward_mean=0.1, reward_bound=0.0
338: loss=0.000, reward_mean=0.1, reward_bound=0.0
339: loss=0.000, reward_mean=0.1, reward_bound=0.0
340: loss=0.000, reward_mean=0.0, reward_bound=0.0
341: loss=0.000, reward_mean=0.0, reward_bound=0.0
342: loss=0.000, reward_mean=0.0, reward_bound=0.0
343: loss=0.000, reward_mean=0.1, reward_bound=0.0
344: loss=0.000, reward_mean=0.1, reward_bound=0.0
345: loss=0.000, reward_mean=0.1, reward_bound=0.0
346: loss=0.000, reward_mean=0.1, reward_bound=0.0
347: loss=0.000, reward_mean=0.1, reward_bound=0.0
348: loss=0.000, reward_mean=0.1, reward_bound=0.0
349: loss=0.000, reward_mean=0.1, reward_bound=0.0
350: loss=0.000, reward_mean=0.

492: loss=0.094, reward_mean=0.0, reward_bound=0.0
493: loss=0.000, reward_mean=0.0, reward_bound=0.0
494: loss=0.000, reward_mean=0.1, reward_bound=0.0
495: loss=0.000, reward_mean=0.0, reward_bound=0.0
496: loss=0.000, reward_mean=0.1, reward_bound=0.0
497: loss=0.000, reward_mean=0.1, reward_bound=0.0
498: loss=0.000, reward_mean=0.1, reward_bound=0.0
499: loss=0.000, reward_mean=0.0, reward_bound=0.0
500: loss=0.000, reward_mean=0.0, reward_bound=0.0
501: loss=0.000, reward_mean=0.1, reward_bound=0.0
502: loss=0.000, reward_mean=0.1, reward_bound=0.0
503: loss=0.000, reward_mean=0.0, reward_bound=0.0
504: loss=0.000, reward_mean=0.0, reward_bound=0.0
505: loss=0.000, reward_mean=0.0, reward_bound=0.0
506: loss=0.000, reward_mean=0.0, reward_bound=0.0
507: loss=0.001, reward_mean=0.0, reward_bound=0.0
508: loss=0.000, reward_mean=0.0, reward_bound=0.0
509: loss=0.066, reward_mean=0.0, reward_bound=0.0
510: loss=0.000, reward_mean=0.0, reward_bound=0.0
511: loss=0.001, reward_mean=0.

660: loss=0.007, reward_mean=0.0, reward_bound=0.0
661: loss=0.072, reward_mean=0.0, reward_bound=0.0
662: loss=0.091, reward_mean=0.1, reward_bound=0.0
663: loss=0.061, reward_mean=0.1, reward_bound=0.0
664: loss=0.004, reward_mean=0.0, reward_bound=0.0
665: loss=0.053, reward_mean=0.0, reward_bound=0.0
666: loss=0.084, reward_mean=0.1, reward_bound=0.0
667: loss=0.181, reward_mean=0.1, reward_bound=0.0
668: loss=0.011, reward_mean=0.0, reward_bound=0.0
669: loss=0.033, reward_mean=0.1, reward_bound=0.0
670: loss=0.017, reward_mean=0.1, reward_bound=0.0
671: loss=0.042, reward_mean=0.0, reward_bound=0.0
672: loss=0.027, reward_mean=0.1, reward_bound=0.0
673: loss=0.078, reward_mean=0.1, reward_bound=0.0
674: loss=0.026, reward_mean=0.1, reward_bound=0.0
675: loss=0.049, reward_mean=0.1, reward_bound=0.0
676: loss=0.001, reward_mean=0.0, reward_bound=0.0
677: loss=0.000, reward_mean=0.0, reward_bound=0.0
678: loss=0.001, reward_mean=0.0, reward_bound=0.0
679: loss=0.007, reward_mean=0.

827: loss=0.004, reward_mean=0.1, reward_bound=0.0
828: loss=0.007, reward_mean=0.1, reward_bound=0.0
829: loss=0.066, reward_mean=0.1, reward_bound=0.0
830: loss=0.005, reward_mean=0.1, reward_bound=0.0
831: loss=0.000, reward_mean=0.0, reward_bound=0.0
832: loss=0.002, reward_mean=0.0, reward_bound=0.0
833: loss=0.006, reward_mean=0.1, reward_bound=0.0
834: loss=0.000, reward_mean=0.0, reward_bound=0.0
835: loss=0.000, reward_mean=0.0, reward_bound=0.0
836: loss=0.029, reward_mean=0.1, reward_bound=0.0
837: loss=0.006, reward_mean=0.1, reward_bound=0.0
838: loss=0.003, reward_mean=0.1, reward_bound=0.0
839: loss=0.002, reward_mean=0.1, reward_bound=0.0
840: loss=0.000, reward_mean=0.0, reward_bound=0.0
841: loss=0.000, reward_mean=0.0, reward_bound=0.0
842: loss=0.000, reward_mean=0.0, reward_bound=0.0
843: loss=0.002, reward_mean=0.1, reward_bound=0.0
844: loss=0.002, reward_mean=0.1, reward_bound=0.0
845: loss=0.004, reward_mean=0.0, reward_bound=0.0
846: loss=0.004, reward_mean=0.

993: loss=0.000, reward_mean=0.1, reward_bound=0.0
994: loss=0.000, reward_mean=0.1, reward_bound=0.0
995: loss=0.000, reward_mean=0.0, reward_bound=0.0
996: loss=0.000, reward_mean=0.0, reward_bound=0.0
997: loss=0.000, reward_mean=0.1, reward_bound=0.0
998: loss=0.000, reward_mean=0.1, reward_bound=0.0
999: loss=0.000, reward_mean=0.0, reward_bound=0.0
1000: loss=0.000, reward_mean=0.0, reward_bound=0.0
1001: loss=0.000, reward_mean=0.0, reward_bound=0.0
1002: loss=0.000, reward_mean=0.0, reward_bound=0.0
1003: loss=0.000, reward_mean=0.0, reward_bound=0.0
1004: loss=0.000, reward_mean=0.0, reward_bound=0.0
1005: loss=0.000, reward_mean=0.0, reward_bound=0.0
1006: loss=0.000, reward_mean=0.1, reward_bound=0.0
1007: loss=0.000, reward_mean=0.1, reward_bound=0.0
1008: loss=0.001, reward_mean=0.1, reward_bound=0.0
1009: loss=0.000, reward_mean=0.1, reward_bound=0.0
1010: loss=0.000, reward_mean=0.0, reward_bound=0.0
1011: loss=0.000, reward_mean=0.0, reward_bound=0.0
1012: loss=0.000, r

1153: loss=0.000, reward_mean=0.0, reward_bound=0.0
1154: loss=0.001, reward_mean=0.2, reward_bound=0.0
1155: loss=0.000, reward_mean=0.0, reward_bound=0.0
1156: loss=0.000, reward_mean=0.1, reward_bound=0.0
1157: loss=0.000, reward_mean=0.1, reward_bound=0.0
1158: loss=0.000, reward_mean=0.0, reward_bound=0.0
1159: loss=0.000, reward_mean=0.0, reward_bound=0.0
1160: loss=0.000, reward_mean=0.1, reward_bound=0.0
1161: loss=0.000, reward_mean=0.1, reward_bound=0.0
1162: loss=0.000, reward_mean=0.0, reward_bound=0.0
1163: loss=0.001, reward_mean=0.1, reward_bound=0.0
1164: loss=0.000, reward_mean=0.1, reward_bound=0.0
1165: loss=0.000, reward_mean=0.0, reward_bound=0.0
1166: loss=0.000, reward_mean=0.0, reward_bound=0.0
1167: loss=0.000, reward_mean=0.1, reward_bound=0.0
1168: loss=0.000, reward_mean=0.0, reward_bound=0.0
1169: loss=0.001, reward_mean=0.2, reward_bound=0.0
1170: loss=0.000, reward_mean=0.0, reward_bound=0.0
1171: loss=0.000, reward_mean=0.1, reward_bound=0.0
1172: loss=0

KeyboardInterrupt: 